# Методы сбора и обработки данных из Интернет

## Урок 1. Практическое задание

### Задание 1

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

**Примечания:**

В домашнем задании о hh.ru ссылку на следующую страницу нужно будет получать из узла. Обработали страницу, получили ссылку на следующую из узла, сделали GET-запрос - у вас страницы и вы ее снова обрабатываете.

Чтобы получить только вакансии, без рекламы - делем поиск по class.
Сначала обрабатываем первую страницу.
Затем вторую и следующие страницы (&page=1 и т.д.). Но так можно попасть на страницу без требуемой информации.

На символе последней страницы код следующий:
<a class="bloko-button HH-Pager-Control" data-page="6" data-qa="pager-page" 
   
Более удобный способ - кнопка "Дальше". Когда дойдем до последней страницы, то кнопки "Дальше" не будет.
Нужно будет проверять на каждой странице наличие кнопки "Дальше". Если она есть, извлекаем атрибут href и идем дальше. Если кнопки "Дальше" нет - все закончено, складываете в общую структуру, выдаете результат.

На superjob.ru такой же подход, только первая страница с цифрой 1, а не 0, как на hh.ru.
На superjob.ru названия class нечеловекочитаемые.

Если даныне о зарплате отсутствуют, то пишите None. Значения зарплаты должны быть типа int.

In [406]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from pprint import pprint

In [407]:
#main_link = 'https://hh.ru/search/vacancy'

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/83.0.4103.106 Safari/537.36'}

In [ ]:
#vacancy = 'Data scientist'

params = {
    'text': vacancy, \
    'L_save_area':True, \
    'clusters':True, \
    'enable_snippets':True, \
    'st':'searchVacancy', \
    'fromSearch':True, \
     'from': 'suggest_post'
}

In [ ]:
#response = requests.get(main_link, params=params, headers=headers)
#soup = bs(response.text,'lxml')

Название вакансии:
<a class="bloko-link HH-LinkModifier" data-qa="vacancy-serp__vacancy-title" href="https://hh.ru/vacancy/37645302?query=Data%20scientist" data-position="0" data-requestid="1593355036276b01eb009967f76652d4" data-totalvacancies="312" target="_blank">Data Scientist</a>

In [ ]:
#page_block = soup.find('div', {'data-qa': 'pager-block'})
#pprint(page_block)

In [ ]:
#last_page = int(page_block.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())
#pprint(last_page)

In [ ]:
#vacancy_items = soup.find('div', {'data-qa': 'vacancy-serp__results'}).find_all('div', {'class': 'vacancy-serp-item'})
#pprint(vacancy_items)

In [ ]:
def _parser_hh(vacancy):

    vacancy_date = []
    
    params = {
        'text': vacancy, \
        'L_save_area':True, \
        'clusters':True, \
        'enable_snippets':True, \
        'st':'searchVacancy', \
        'fromSearch':True, \
         'from': 'suggest_post', \
        'page': ''
}
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                AppleWebKit/537.36 (KHTML, like Gecko) \
                Chrome/83.0.4103.106 Safari/537.36'}

    main_link = 'https://hh.ru/search/vacancy'
       
    response = requests.get(main_link, params=params, headers=headers)
    
    if response.ok:
        soup = bs(response.text,'lxml')
        
        page_block = soup.find('div', {'data-qa': 'pager-block'})
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())
    
    for page in range(0, last_page):
        params['page'] = page
        response = requests.get(main_link, params=params, headers=headers)
        
        if response.ok:
            soup = bs(response.text,'lxml')
            
            vacancy_items = soup.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
                
            for item in vacancy_items:
                vacancy_date.append(_parser_item_hh(item))
                
    return vacancy_date

"vacancy-serp__vacancy-compensation">от 150&nbsp;000 руб.

class="vacancy-serp-item__controls-item vacancy-serp-item__controls-item_response


In [ ]:
#vacancy_name = soup.find('span', {'class': 'resume-search-item__name'})
#pprint(vacancy_name)

data-qa="vacancy-serp__vacancy-compensation"

In [ ]:
#salary = soup.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
#pprint(salary)

In [ ]:
#is_ad = soup.find('a', {'data-qa': 'vacancy-serp__vacancy_response'})
#pprint(is_ad)

In [ ]:
#vacancy_link = soup.find('span', {'class': 'resume-search-item__name'}).find('a')['href']
#pprint(vacancy_link)

a href="/applicant/vacancy_response?vacancyId=37635578&amp;hhtmFrom=vacancy_search_result" data-qa="vacancy-serp__vacancy_response" class="bloko-link bloko-link_dimmed HH-VacancyResponsePopup-Link ">Откликнуться

In [ ]:
def _parser_item_hh(item):

    vacancy_data = {}
    
    # vacancy_name
    vacancy_name = item.find('span', {'class': 'resume-search-item__name'}) \
                        .getText() \
                        .replace(u'\xa0', u' ')
    
    vacancy_data['vacancy_name'] = vacancy_name
    
    # company_name
    company_name = item.find('div', {'class': 'vacancy-serp-item__meta-info'}) \
                        .find('a') \
                        .getText()
    
    vacancy_data['company_name'] = company_name
    
    # city
    city = item.find('span', {'class': 'vacancy-serp-item__meta-info'}) \
                .getText() \
                .split(', ')[0]
    
    vacancy_data['city'] = city
      
    #salary
    salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText() \
                        .replace(u'\xa0', u'')
        
        salary = re.split(r'\s|-', salary)
        
        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
        else:
            salary_min = int(salary[0])
            salary_max = int(salary[1])            
        
        salary_currency = salary[2]
        
    vacancy_data['salary_min'] = salary_min
    vacancy_data['salary_max'] = salary_max
    vacancy_data['salary_currency'] = salary_currency
    
    # link
    is_ad = item.find('a', {'data-qa': 'vacancy-serp__vacancy_response'}) \
                .getText()
    
    vacancy_link = item.find('span', {'class': 'resume-search-item__name'}) \
                        .find('a')['href']
    
    if is_ad != 'Реклама':
        vacancy_link = vacancy_link.split('?')[0]
    
    vacancy_data['vacancy_link'] = vacancy_link 
    
    # site
    vacancy_data['site'] = 'hh.ru'
    
    return vacancy_data

main_link = 'https://russia.superjob.ru/vakansii/'

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
             AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/83.0.4103.106 Safari/537.36'}

params = {
    'keywords': vacancy, \
    'page': ''
}

vacancy = 'Аналитик'

response = requests.get(main_link, params=params, headers=headers)
soup = bs(response.text,'lxml')

In [409]:
def parser_vacancy(vacancy):
        
    vacancy_data = []
    vacancy_data.extend(_parser_hh(vacancy))
#    vacancy_data.extend(_parser_superjob(vacancy))
    
    df = pd.DataFrame(vacancy_data)

    return df

In [410]:
vacancy = 'Data scientist'
df = parser_vacancy(vacancy)

In [412]:
df[100:150]

,vacancy_name,company_name,city,salary_min,salary_max,salary_currency,vacancy_link,site
100,Ведущий инженер по тестированию,ООО UMA.Tech,Москва,NaN,250000.0,руб.,https://hh.ru/vacancy/36057559,hh.ru
101,Chief Data Scientist,Аэропорт Пулково (ООО Воздушные Ворота Северн...,Санкт-Петербург,NaN,NaN,None,https://hh.ru/vacancy/36971316,hh.ru
102,Эксперт по математическому моделированию,"Кселл, АО",Алматы,NaN,NaN,None,https://hh.ru/vacancy/37229576,hh.ru
103,Старший специалист по математическому моделиро...,"Кселл, АО",Алматы,NaN,NaN,None,https://hh.ru/vacancy/37229561,hh.ru
104,Data scientist (middle),Открытые Технологии,Москва,NaN,150000.0,руб.,https://hh.ru/vacancy/36955232,hh.ru
105,"R&D Team Lead (Industrial Simulation, Data Ana...",Sensoleak LTD,Москва,NaN,NaN,None,https://hh.ru/vacancy/37662636,hh.ru
106,Senior Software Engineer (Python/Devops) for a...,Assaia International AG,Санкт-Петербург,2500.0,3500.0,USD,https://hh.ru/vacancy/37675980,hh.ru
107,Oracle developer,ООО Лайфтех,Минск,NaN,NaN,None,https://hh.ru/vacancy/37468936,hh.ru
108,Senior Data Engineer / Ведущий инженер данных,ООО UMA.Tech,Москва,NaN,290000.0,руб.,https://hh.ru/vacancy/37167337,hh.ru
109,Middle / Senior Data Scientist,Сбербанк для экспертов,Москва,NaN,NaN,None,https://hh.ru/vacancy/36313639,hh.ru
